In [ ]:
(require '[clojupyter.widgets.ipywidgets :as ipy])
(require '[clojure.pprint :refer [pprint]]);

## Coercions
The widget below is actually valid, with the maximum value being limited to 200.

In [ ]:
(def widg0 (ipy/bounded-int-text {:min 10 :value 30 :max "200"}))
widg0

The expression bellow returns true for python kernel. Where numbers are expected they are automatically parsed if strings are passed to arguments.

The same is not true for disabled argument, i.e. ipy.BoundedIntText(disabled="True") fails, so it looks like the statement above is an implementation detail.

In [ ]:
(= (:max @(ipy/bounded-int-text {:max 300}))
   (:max @(ipy/bounded-int-text {:max "300"})))

The expression bellow throws an exception in python. ~~In clojupyter, it returns an invalid widget model. It looks like a widget, but it's unusable (it's value cannot be set)~~

FIXED: :min, :max and :value cannot be set to strings in clojupyter.

In [ ]:
(ipy/bounded-int-text {:max "some big number"})

In [ ]:
(ipy/int-slider {:style (ipy/slider-style {:handle_color :green})})

## Style and Layout
In python, when a widget model is created, a new instance of Layout and Style are created and referenced in the model.
Currently, in clojupyter those are set to nil.

In [ ]:
(select-keys @(ipy/textarea) [:layout :style])

We can instantiate those values manually

In [ ]:
(select-keys @(ipy/textarea {:layout (ipy/layout) :style (ipy/description-style)}) [:layout :style])

In [ ]:
(ipy/textarea {:rows 5 :description "Multiline Text Description" :placeholder "Here be dragons" :layout (ipy/layout {:width "800px" :border "green solid"}) :style (ipy/description-style {:description_width "initial"})})

*:layout* attribute expects only a layout widget.
*:style* attribute expects one of several style widgets.

These two references can be passed directly under the hood, to allow users to define the widget directly as:

In [ ]:
(ipy/textarea {:rows 5 :description "Multiline Text Description" :placeholder "Here be dragons" :layout {:width "800px" :border "green solid"} :style {:description_width "initial"}})

## Private Attributes
Attributes that start with **_** look like private.
Does it serve any porpuse to allow the end user to change them?

In [ ]:
(def mutant-widget (ipy/bounded-int-text {:_model_name "FloatSliderModel" :_view_name "FloatSliderView"}))
mutant-widget

## Complex widgets
The example bellow contains a horizontal box widget with two other widgets, an int slider and an int progress bar. The two widgets are linked by a directional link, that updates the progress's bar :value automatically, when the int slider :value changes.

In [ ]:
(def SSP
    (let [slider-style (ipy/slider-style {:handle_color "tomato"})
      w (ipy/int-slider {:orientation "vertical" :value (rand-int 101) :style slider-style :description "x"})
      ww (ipy/int-slider {:orientation "vertical" :value (* 2 (:value @w)) :max (* 2 (:max @w)) :style slider-style :description "2*x" :disabled true})
      _ (.watch w :double (fn [_ _ _ {value :value}] (swap! ww assoc :value (* 2 value))))
      p (ipy/int-progress {:orientation "vertical" :value (:value @w) :bar_style "danger"})
      pv (ipy/label {:value (str (:value @p)) :_dom_classes ["output"]})
      _ (.watch p :print (fn [_ _ _ {value :value}] (swap! pv assoc :value (str value))))
      _ (ipy/directional-link {:source [w :value] :target [p :value]})
      ppv (ipy/v-box {:children [p pv] :layout (ipy/layout {:align_items "center"})})]
  (ipy/h-box {:children [w ww ppv]})))
SSP

In [ ]:
(->> SSP deref :children last deref :children first deref :value)

## Issues
### The Problem with Big Numbers
Neither python, nor clojure have a problem handling big integers. The widget runs in a JavaScript context and JavaScript truncates automatically integers with more than 16 digits.

E.g: The widget below shows a default value (5.555555555555555e+29) that's different than the underlying model value (555555555555555555555555555555).

Practically, that means our integer widgets have a (undocumented) maximum/minimum value.

In [ ]:
(def widg1 (ipy/bounded-int-text {:value 555555555555555555555555555555 :max 9999999999999999999999999999999999999999999999999999999999999999999999}))
widg1

In [ ]:
(= (:max @widg1) 9999999999999999999999999999999999999999999999999999999999999999999999)

In [ ]:
(= (:value @widg1) 555555555555555555555555555555)

### Float Widgets
Float widgets send an integer value when it hits one (7 instead of 7.0)

In [ ]:
(ipy/float-log-slider {:value 2e5 :base 10.0 :min 4.0 :max 8.0 :step 0.2 :continuous_update false})

In [ ]:
(ipy/float-slider)

In [ ]:
(ipy/float-progress)

In [ ]:
(ipy/float-text)

### File Uploader
File upload widget does not return the file contents, only its metadata.

In [ ]:
(def WU (ipy/file-upload))
WU

In [ ]:
@WU

### Date Picker
Date-picker widget returns a hash-map.Should it return a java.util.Date or java.time.LocalDate ?

In [ ]:
(def date-widget (ipy/date-picker))
date-widget

In [ ]:
@date-widget